In [ ]:
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import json
import time

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm_notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_train.csv', index_col=[0])
X_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/X_test.csv', index_col=[0])
y_train = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_train.csv', index_col=[0])
y_test = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/y_test.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

In [ ]:
X_train_split = X_train['댓글'].apply(list).tolist()
X_test_split = X_test['댓글'].apply(list).tolist()

In [ ]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

X_train_sequences = tokenizer.texts_to_sequences(X_train_split)
X_test_sequences = tokenizer.texts_to_sequences(X_test_split)

In [ ]:
train = pad_sequences(X_train_sequences, padding='post', maxlen=400)
test = pad_sequences(X_test_sequences, padding='post', maxlen=400)

In [ ]:
y_train.value_counts()

악플여부
0       55644
1       23944
dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
start = time.time()

smote = SMOTE(random_state=123)
train_over, y_train_over = smote.fit_sample(train, y_train)

print("걸린시간 :", time.time() - start)

걸린시간 : 241.5743203163147


In [ ]:
# train_over = train
# y_train_over = y_train

In [ ]:
import tensorflow.keras.backend as K
# Mish 활성화 함수
def mish(x):
    return x * K.tanh(K.softplus(x))

In [ ]:
# 음절 단어사전의 크기
vocab_size = len(word_index_vocab)
embedding_dim = 32
comment_len = 400

In [34]:
def get_model() :

    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Flatten, MaxPool1D, GlobalAveragePooling1D, Flatten


    model = Sequential()
    model.add(Embedding(vocab_size+1, embedding_dim, input_length=comment_len))

    model.add(Dropout(0.2))
    model.add(Conv1D(32, 3, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.003), metrics=['accuracy'])

    return model


In [35]:
model = get_model()

In [36]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
mc = ModelCheckpoint('1D_CNN_best.h5', monitor='val_loss', mode='min', save_best_only=True)

In [37]:
history = model.fit(train_over, y_train_over, 
                    callbacks        = [es, mc],
                    epochs           = 50,
                    batch_size       = 32, 
                    validation_data = (test, y_test))

Epoch 1/50
3478/3478 [==============================] - 68s 19ms/step - loss: 0.3539 - accuracy: 0.8434 - val_loss: 0.2697 - val_accuracy: 0.8957
Epoch 2/50
3478/3478 [==============================] - 68s 20ms/step - loss: 0.2766 - accuracy: 0.8852 - val_loss: 0.2560 - val_accuracy: 0.9009
Epoch 3/50
3478/3478 [==============================] - 67s 19ms/step - loss: 0.2524 - accuracy: 0.8967 - val_loss: 0.2535 - val_accuracy: 0.9016
Epoch 4/50
3478/3478 [==============================] - 68s 19ms/step - loss: 0.2419 - accuracy: 0.9008 - val_loss: 0.2415 - val_accuracy: 0.9069
Epoch 5/50
3478/3478 [==============================] - 68s 20ms/step - loss: 0.2323 - accuracy: 0.9060 - val_loss: 0.2415 - val_accuracy: 0.9066
Epoch 6/50
3478/3478 [==============================] - 73s 21ms/step - loss: 0.2253 - accuracy: 0.9082 - val_loss: 0.2419 - val_accuracy: 0.9075
Epoch 7/50
3478/3478 [==============================] - 67s 19ms/step - loss: 0.2196 - accuracy: 0.9111 - val_loss: 0.2406 -

In [38]:
# 저장한 CNN 모델을 불러오고 예측해보자


loaded_model = load_model('1D_CNN_best.h5')
y_pred = loaded_model.predict(test)

y_class = y_pred > 0.5

In [39]:
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix

print('recall : ', recall_score(y_class, y_test))
print('precision : ', precision_score(y_class, y_test))
print('f1_score : ', f1_score(y_class, y_test))
print('roc_auc_score : ', roc_auc_score(y_test, y_pred))
print(confusion_matrix(y_class, y_test))

recall :  0.8747776591960157
precision :  0.8174867021276596
f1_score :  0.8451623990376355
roc_auc_score :  0.9551234888650796
[[13177  1098]
 [  704  4918]]


In [ ]:
compare_df = pd.DataFrame({
    '댓글' : np.array(X_test).reshape(1, -1)[0],
    '예측' : y_class.reshape(1, -1)[0],
    '실제' : y_test.values.reshape(1, -1)[0]
})

compare_df['예측'] = compare_df['예측'].apply(int)

In [ ]:
compare_df[compare_df['예측'] != compare_df['실제']].to_csv('/content/drive/MyDrive/[final_project]_악플원정대/no_match.csv')

In [ ]:
def model_test(comment):
    loaded_model = load_model('1D_CNN_best.h5')
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)

    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)

    pred = loaded_model.predict(comment_pad)

    if pred < 0.5 :
        result = '악플이 아닙니다'
    else :
        result = '악플입니다'
    
    print(result)
    print('라벨확률 : ', pred)
    
    return result, pred

In [33]:
comment = '''왜 나는 아무것도 안되지'''

result, prob = model_test(comment)

악플이 아닙니다
라벨확률 :  [[0.08028197]]
